In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(dblyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in library(dblyr): there is no package called ‘dblyr’


In [3]:
bone_marrow <- read_csv("https://raw.githubusercontent.com/rishiiii3/dsci-100-2023S-group-67/main/bone-marrow.arff", 
                        skip = 108, col_names = FALSE)
bone_marrow

Rows: 187 Columns: 37
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (13): X8, X9, X10, X11, X12, X13, X14, X20, X21, X28, X30, X31, X32
dbl (24): X1, X2, X3, X4, X5, X6, X7, X15, X16, X17, X18, X19, X22, X23, X24...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,22.83014,0,1,0,1,1,1,0,⋯,1,7.20,1.33876,5.38,35,19,51,32,999,0
1,0,23.34247,0,1,0,-1,-1,1,0,⋯,1,4.50,11.078295,0.41,20.6,16,37,1000000,163,1
1,0,26.39452,0,1,0,-1,-1,1,0,⋯,1,7.94,19.01323,0.42,23.4,23,20,1000000,435,1
0,0,39.68493,1,1,0,1,2,1,1,⋯,?,4.25,29.481647,0.14,50,23,29,19,53,1
0,1,33.35890,0,0,0,1,2,0,1,⋯,1,51.85,3.972255,13.05,9,14,14,1000000,2043,0
1,0,27.39178,0,0,0,2,0,1,1,⋯,1,3.27,8.412758,0.39,40,16,70,1000000,2800,0
0,1,34.52055,0,1,0,0,1,0,1,⋯,?,17.78,2.406248,7.39,51,17,29,18,41,1
1,0,21.43562,0,1,0,0,1,1,1,⋯,?,6.41,?,?,56,22,58,22,45,1
1,1,32.64110,0,0,0,2,0,1,1,⋯,1,23.54,3.772555,6.24,20.5,15,14,1000000,671,0


In [19]:
bone_marrow_selected <- select(bone_marrow, X3, X23, X29, X32,X37) |> 
rename(  Donor_Age = X3 ,
         Recipient_Age = X23,
         CD34_Dose = X29,
         Body_Mass = X32,
         Survival_Status = X37) |>
mutate(Survival_Status = as_factor(Survival_Status)) |>
mutate(Body_Mass = as.numeric(Body_Mass))
bone_marrow_selected[bone_marrow_selected == "?"] <- "NA"
bone_marrow_selected |> is.na()|> sum()

#filter(!(is.na(bone_marrow_selected)))

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


[1] 2

There are 2 rows that have missing data

In [21]:
bone_marrow_clean <- bone_marrow_selected |> drop_na(Donor_Age:Survival_Status)
bone_marrow_clean

Donor_Age,Recipient_Age,CD34_Dose,Body_Mass,Survival_Status
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
22.83014,9.6,7.20,35.0,0
23.34247,4.0,4.50,20.6,1
26.39452,6.6,7.94,23.4,1
39.68493,18.1,4.25,50.0,1
33.35890,1.3,51.85,9.0,0
27.39178,8.9,3.27,40.0,0
34.52055,14.4,17.78,51.0,1
21.43562,18.2,6.41,56.0,1
32.64110,7.9,23.54,20.5,0


The data above is in a tidyform 

In [22]:

mean_predictors <- bone_marrow_clean |> 
summarize(across(Donor_Age:Body_Mass, mean, na.rm = TRUE))
mean_predictors

Donor_Age,Recipient_Age,CD34_Dose,Body_Mass
<dbl>,<dbl>,<dbl>,<dbl>
33.45004,9.912973,11.78585,35.80108


Mean values for each predictor in the dataset

In [18]:
obs_per_class <- bone_marrow_clean |> group_by(Survival_Status) |> summarise( count=n())
obs_per_class


Survival_Status,count
<fct>,<int>
0,102
1,83


The number of observations for each class where:
0 represents deceased and 1 represents survived